In [36]:
import pandas as pd

In [37]:
FILE = "/Users/damoncrockett/vosd.org/215-opendsd/tables/approval_master_FULL.csv"

In [38]:
df = pd.read_csv(FILE,low_memory=False)

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 388645 entries, 0 to 388644
Data columns (total 30 columns):
Unnamed: 0               388645 non-null int64
APN                      360762 non-null object
ApplicationDate          388645 non-null object
ApplicationExpiration    246135 non-null object
ApplicationExpired       388645 non-null bool
ApplicationMonth         388645 non-null int64
ApplicationYear          388645 non-null int64
ApprovalId               388645 non-null int64
Description              388645 non-null object
DevelopmentId            388645 non-null int64
DevelopmentTitle         388645 non-null object
IssueDate                338383 non-null object
IssueMonth               338383 non-null float64
IssueYear                338383 non-null float64
IssuedBy                 338383 non-null object
JobId                    388645 non-null int64
Latitude                 388585 non-null float64
Longitude                388585 non-null float64
PermitHolder             3886

In [40]:
df.Type.value_counts().head(15)

Electrical Permit                 88354
Plumbing Permit                   80861
Combination Building Permit       56166
Mechanical Permit                 53193
Building Permit                   40150
Electrical-Photovoltaic           10390
Construction Change - Building     9414
Sign Permit                        7896
Right Of Way Permit                7149
Right Of Way Permit-Const Plan     4663
Encroachment Agreement             4316
Fire Permit                        2364
Fire Permit - Suppression          2336
Fire Permit - Alarm                1800
Coastal Development Permit         1694
dtype: int64

In [41]:
df = df[df.Type=="Right Of Way Permit"]

In [42]:
import numpy as np

In [43]:
np.median(df.TimetoIssue)

8.0

In [44]:
d = df[['ApprovalId','APN','ProjectId','Status','TimetoIssue','ApplicationYear']]

In [45]:
statuses = ['Issued','Completed']
relevant = d[d.Status.isin(statuses)]

In [46]:
times = list(relevant.TimetoIssue)

In [47]:
np.mean(times)

88.29696556426866

In [48]:
from collections import Counter

In [49]:
len(times)

5866

In [50]:
pd.DataFrame.from_dict(Counter(times).items()).head()

,0,1
0,0,3069
1,1,212
2,2,61
3,3,44
4,4,43


In [51]:
d.reset_index(drop=True,inplace=True)

In [52]:
d.to_csv("/Users/damoncrockett/vosd.org/215-opendsd/tables/ROW.csv",index=False)

In [66]:
len(d.ApprovalId)

7149

###Fees (from invoice tables)

In [53]:
DATA = "/Users/damoncrockett/vosd.org/215-opendsd/tables/invoice_table_basic.csv"

In [54]:
di = pd.read_csv(DATA)

In [55]:
paid = di[di.InvoiceStatus=='Paid']

In [56]:
projects = list(set(list(d.ProjectId)))

In [57]:
paid[paid.ProjectId.isin(projects)].head()

,InvoiceId,InvoiceStatus,ProjectId,TotalAmount,PaidDate
85,287687,Paid,2814,2848.00,2009-03-05T11:23:21
86,110783,Paid,2814,1700.00,2005-07-20T15:57:29
87,224188,Paid,2814,3834.00,2007-10-11T11:26:44
107,232987,Paid,3531,5000.00,2007-12-19T10:50:50
110,115041,Paid,3531,464.13,2005-08-17T13:41:41


In [58]:
# problem: these are the total invoice amounts from projects that involve ROW permits
# We need how much the ROW permits contribute to this total

###New Tack: use the oft-maligned approval database

In [59]:
import sqlite3
import json

In [60]:
con = sqlite3.connect('/Users/damoncrockett/Dropbox/thinkpad_desktop_2/DSD_sqlite_db/approvals.db')
cur = con.cursor()

In [82]:
approvals = list(set(list(relevant.ApprovalId)))

In [68]:
counter = 0

for row in cur.execute('SELECT data FROM json'):
    r = json.loads(str(row[0]).decode('zlib'))
    if 'ApprovalId' in r and r['ApprovalId']:
        if r['ApprovalId'] in approvals:
            counter +=1
print counter

832


In [69]:
# okay: maligned for good reason. We are missing like 5,000 permits here

###Getting approval fees from invoices

In [70]:
con = sqlite3.connect('/Users/damoncrockett/Dropbox/thinkpad_desktop_2/DSD_sqlite_db/invoices.db')
cur = con.cursor()

In [71]:
counter = 0

for row in cur.execute('SELECT data FROM json'):
    counter +=1
print counter

474190


In [74]:
counter = 0
l = []

for row in cur.execute('SELECT data FROM json'):
    r = json.loads(str(row[0]).decode('zlib'))
    if 'ProjectDetails' in r and r['ProjectDetails']:
        counter +=1
        n = len(r['ProjectDetails'])
        l.append(n)
print counter, Counter(l)

472706 Counter({1: 468508, 2: 2391, 3: 644, 4: 362, 5: 242, 6: 128, 7: 85, 8: 80, 9: 53, 10: 31, 11: 27, 13: 17, 12: 13, 16: 11, 19: 11, 20: 11, 26: 10, 14: 9, 18: 9, 21: 9, 15: 6, 17: 6, 25: 5, 23: 4, 22: 3, 33: 3, 36: 3, 58: 3, 63: 3, 24: 2, 29: 2, 39: 2, 59: 2, 62: 2, 27: 1, 28: 1, 32: 1, 42: 1, 44: 1, 46: 1, 47: 1, 60: 1, 72: 1})


In [75]:
ids = []
amts = []
auths = []
descr = []

for row in cur.execute('SELECT data FROM json'):
    r = json.loads(str(row[0]).decode('zlib'))
    if 'ProjectDetails' in r and r['ProjectDetails']:
        tmp = r['ProjectDetails']
        n = len(tmp)
        for i in range(n):
            if 'Jobs' in tmp[i] and tmp[i]['Jobs']:
                tmp2 = tmp[i]['Jobs']
                m = len(tmp2)
                for i in range(m):
                    if 'ApprovalInfo' in tmp2[i] and tmp2[i]['ApprovalInfo']:
                        tmp3 = tmp2[i]['ApprovalInfo']
                        k = len(tmp3)
                        for i in range(k):
                            if 'ApprovalFees' in tmp3[i] and tmp3[i]['ApprovalFees']:
                                tmp4 = tmp3[i]['ApprovalFees']
                                l = len(tmp4)
                                for i in range(l):
                                    if 'ApprovalId' in tmp4[i] and tmp4[i]['ApprovalId']:
                                        ids.append(tmp4[i]['ApprovalId'])
                                    else:
                                        ids.append('missing')
                                    if 'FeeAmount' in tmp4[i] and tmp4[i]['FeeAmount']:
                                        amts.append(tmp4[i]['FeeAmount'])
                                    else:
                                        amts.append('missing')
                                    if 'FeeAuthority' in tmp4[i] and tmp4[i]['FeeAuthority']:
                                        auths.append(tmp4[i]['FeeAuthority'])
                                    else:
                                        auths.append('missing')
                                    if 'FeeDescription' in tmp4[i] and tmp4[i]['FeeDescription']:
                                        descr.append(tmp4[i]['FeeDescription'])
                                    else:
                                        descr.append('missing')

In [76]:
print len(ids), len(amts), len(auths), len(descr)

3228122 3228122 3228122 3228122


In [77]:
FEES = pd.DataFrame(ids,columns=['ApprovalId'])

In [79]:
FEES['FeeAmount'] = amts
FEES['FeeAuthority'] = auths
FEES['FeeDescription'] = descr

In [81]:
FEES.to_csv("/Users/damoncrockett/vosd.org/215-opendsd/tables/approval_fees_complete.csv",
            index=False,
           encoding='utf-8')

####Okay sweet! Now to check for specifically ROW permits...

In [83]:
FEES_ROW = FEES[FEES.ApprovalId.isin(approvals)]

In [90]:
FEES_ROW.reset_index(drop=True,inplace=True)

In [91]:
type(FEES_ROW.FeeAmount.loc[0])

float

In [92]:
FEES_ROW = FEES_ROW[FEES_ROW.FeeAmount != 'missing']

In [102]:
len(FEES_ROW.index)

14681

In [95]:
tmp = FEES_ROW.groupby('ApprovalId').agg(np.sum)[['FeeAmount']]

In [98]:
np.mean(tmp.FeeAmount)

1914.5286204837264

In [99]:
np.median(tmp.FeeAmount)

586.0

In [101]:
len(tmp.index)

3349